# Imports

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn. metrics import roc_auc_score, accuracy_score
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve,roc_auc_score

In [2]:
train_1M=pd.read_csv('../data/train_1M_with_features_v1.csv')
questions=pd.read_csv('../data/questions.csv')

# B0 model

In [3]:
X=train_1M.query('content_type_id==0')[['qstats_answered_correctly','user_avg_score_cum']]
y=train_1M.query('content_type_id==0')['answered_correctly']

In [4]:
log = LogisticRegression(solver='lbfgs', max_iter=1000)
y_predic=cross_val_predict(log,X,y,method='predict_proba',cv=10,verbose=1,)
y_predic_proba1=y_predic[:,1]  #on prend la 2e colonne, celle des probas de la classe 1
roc_auc_score(y,y_predic_proba1)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   16.7s finished


0.7467683128847293

In [5]:
log.fit(X,y)
import pickle
pickle.dump(log, open("B0model.pkl", "wb") )

In [6]:
X

,qstats_answered_correctly,user_avg_score_cum
0,0.8370,0.000000
1,0.3972,1.000000
2,0.3640,0.500000
3,0.6514,0.333333
4,0.1749,0.250000
...,...,...
987264,0.5967,0.600000
987265,0.5063,0.615385
987266,0.4424,0.629630
987267,0.2932,0.642857


# B0 ++ avec qq features

In [11]:
selected_features=['qstats_answered_correctly','user_avg_score_cum','already_seen','user_personalized_qstat_knowing_had_explanation_or_not','at_least_one_lesson']
X=train_1M.query('content_type_id==0')[selected_features]
y=train_1M.query('content_type_id==0')['answered_correctly']
X.isna().sum()

qstats_answered_correctly                                 0
user_avg_score_cum                                        0
already_seen                                              0
user_personalized_qstat_knowing_had_explanation_or_not    0
at_least_one_lesson                                       0
dtype: int64

In [12]:
log = LogisticRegression(solver='lbfgs', max_iter=1000)
y_predic=cross_val_predict(log,X,y,method='predict_proba',cv=10)
y_predic_proba1=y_predic[:,1]  #on prend la 2e colonne, celle des probas de la classe 1
roc_auc_score(y,y_predic_proba1)

0.7575087237925182

# Tentative de reproduire le nb kaggle

In [12]:
train_1M = train_1M[train_1M['content_type_id']==0]

In [15]:
question_part=questions.groupby('question_id')[['part']].mean()
question_part.columns = ['content_part']
question_part.reset_index(inplace=True)
question_part = question_part.rename(columns={'question_id':'content_id'})
question_part
del questions

In [17]:
train= pd.merge(train_1M, question_part, on=['content_id'], how="left")

In [18]:
part_grouped = train[['user_id','content_part','answered_correctly']].groupby(['user_id','content_part']).agg(['mean'])
part_grouped.columns = ['part_grouped_mean']
part_grouped

part_grouped_mean
user_id    content_part                   
1710599    2                      0.333333
           5                      0.312500
1906069    2                      1.000000
           5                      0.437500
2101969    2                      0.674074
...                                    ...
2145920530 5                      0.477273
           6                      0.250000
           7                      0.250000
2147379374 2                      0.100000
           5                      0.419355

[18409 rows x 1 columns]

In [20]:
train_df = pd.merge(train,part_grouped, on  =['user_id','content_part'],how="left")

In [27]:
train_df

,Unnamed: 0,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,...,qstats_timestamp_knowing_answered_correctly,qstats_user_activity_cumcount_knowing_answered_correctly,qstats_prior_question_elapsed_time_knowing_answered_correctly,qstats_task_container_id_knowing_answered_uncorrectly,qstats_timestamp_knowing_answered_uncorrectly,qstats_user_activity_cumcount_knowing_answered_uncorrectly,qstats_prior_question_elapsed_time_knowing_answered_uncorrectly,user_personalized_qstat_knowing_had_explanation_or_not,content_part,part_grouped_mean
0,0,90286654,0,1917429896,7900,0,0,1,NaN,0,...,inf,32.60,20780.0,37.10,inf,54.10,23140.0,0.836426,1,0.333333
1,1,90286655,26511,1917429896,7876,0,1,0,21000.0,0,...,inf,107.44,21870.0,34.00,inf,44.25,22600.0,0.376953,1,0.333333
2,2,90286656,55312,1917429896,175,0,2,0,21000.0,0,...,inf,107.06,23550.0,55.03,inf,71.70,23180.0,0.351318,1,0.333333
3,3,90286657,102436,1917429896,1278,0,3,0,23000.0,0,...,inf,66.70,23360.0,11.08,inf,12.72,23140.0,0.645020,2,0.000000
4,4,90286658,188454,1917429896,2063,0,4,0,45000.0,0,...,inf,150.50,18350.0,55.20,inf,77.25,18990.0,0.164185,3,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967829,987264,47484293,1619704,1003849974,6909,0,14,1,17000.0,0,...,inf,271.20,28030.0,151.40,inf,220.10,26110.0,0.592773,6,1.000000
967830,987265,47484294,1906094,1003849974,7217,0,15,1,68500.0,0,...,inf,128.80,34800.0,40.47,inf,64.94,31120.0,0.485352,7,1.000000
967831,987266,47484295,1906094,1003849974,7216,0,15,1,68500.0,0,...,inf,129.80,34270.0,46.72,inf,71.50,31950.0,0.427002,7,1.000000
967832,987267,47484296,1906094,1003849974,7219,0,15,1,68500.0,0,...,inf,110.80,35200.0,58.84,inf,91.70,32060.0,0.284180,7,1.000000


In [30]:
train_df.query('user_id==1917429896')

,Unnamed: 0,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,...,qstats_timestamp_knowing_answered_correctly,qstats_user_activity_cumcount_knowing_answered_correctly,qstats_prior_question_elapsed_time_knowing_answered_correctly,qstats_task_container_id_knowing_answered_uncorrectly,qstats_timestamp_knowing_answered_uncorrectly,qstats_user_activity_cumcount_knowing_answered_uncorrectly,qstats_prior_question_elapsed_time_knowing_answered_uncorrectly,user_personalized_qstat_knowing_had_explanation_or_not,content_part,part_grouped_mean
0,0,90286654,0,1917429896,7900,0,0,1,NaN,0,...,inf,32.60,20780.0,37.10,inf,54.10,23140.0,0.836426,1,0.333333
1,1,90286655,26511,1917429896,7876,0,1,0,21000.0,0,...,inf,107.44,21870.0,34.00,inf,44.25,22600.0,0.376953,1,0.333333
2,2,90286656,55312,1917429896,175,0,2,0,21000.0,0,...,inf,107.06,23550.0,55.03,inf,71.70,23180.0,0.351318,1,0.333333
3,3,90286657,102436,1917429896,1278,0,3,0,23000.0,0,...,inf,66.70,23360.0,11.08,inf,12.72,23140.0,0.645020,2,0.000000
4,4,90286658,188454,1917429896,2063,0,4,0,45000.0,0,...,inf,150.50,18350.0,55.20,inf,77.25,18990.0,0.164185,3,0.333333
5,5,90286659,188454,1917429896,2065,0,4,1,45000.0,0,...,inf,110.70,19060.0,41.28,inf,54.10,18560.0,0.627441,3,0.333333
6,6,90286660,188454,1917429896,2064,0,4,0,45000.0,0,...,inf,117.06,19220.0,31.62,inf,40.53,18260.0,0.629395,3,0.333333
7,7,90286661,264296,1917429896,3365,0,5,0,26666.0,0,...,inf,127.10,25820.0,51.70,inf,74.00,25380.0,0.293945,4,0.222222
8,8,90286662,264296,1917429896,3364,0,5,1,26666.0,0,...,inf,123.00,25500.0,39.60,inf,52.75,25520.0,0.514648,4,0.222222
9,9,90286663,264296,1917429896,3363,0,5,0,26666.0,0,...,inf,137.00,25760.0,51.94,inf,71.80,25420.0,0.270752,4,0.222222


In [29]:
train_df.loc[train_df.user_id==115]

,Unnamed: 0,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,...,qstats_timestamp_knowing_answered_correctly,qstats_user_activity_cumcount_knowing_answered_correctly,qstats_prior_question_elapsed_time_knowing_answered_correctly,qstats_task_container_id_knowing_answered_uncorrectly,qstats_timestamp_knowing_answered_uncorrectly,qstats_user_activity_cumcount_knowing_answered_uncorrectly,qstats_prior_question_elapsed_time_knowing_answered_uncorrectly,user_personalized_qstat_knowing_had_explanation_or_not,content_part,part_grouped_mean
